In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import os, sys

In [3]:
import pylab as plt
import matplotlib

import numpy as np
import pandas as pd

In [4]:
sys.path.append('..')

from pyMultiOmics.constants import *
from pyMultiOmics.mapping import Mapper
from pyMultiOmics.common import set_log_level_info, set_log_level_debug

2021-01-29 15:02:04.707 | INFO     | pyMultiOmics.reactome:get_neo4j_driver:24 - Created graph database driver for bolt://localhost:7687 (neo4j)


# Demonstration of pyMultiOmics mapping

## Load the processed Covid data from [1]

[1] [Shen, Bo, et al. "Proteomic and metabolomic characterization of COVID-19 patient sera." Cell 182.1 (2020): 59-72.](https://www.sciencedirect.com/science/article/pii/S0092867420306279?casa_token=wKwWn9P4MK0AAAAA:v8z5MVnQ1ONrcatncCsigSDoxeOq2ZOcN4da9SofGDcpeDqrC76QK8yNKrKtVUrMWBBAntI8)

In [5]:
DATA_FOLDER = os.path.abspath(os.path.join('test_data', 'covid19_data'))
DATA_FOLDER

'C:\\Users\\joewa\\Work\\git\\pyMultiOmics\\notebooks\\test_data\\covid19_data'

Read proteomics data

In [6]:
protein_data = pd.read_csv(os.path.join(DATA_FOLDER, 'protein_data.csv'), index_col='Identifier')
protein_design = pd.read_csv(os.path.join(DATA_FOLDER, 'protein_design.csv'), index_col='sample')

In [7]:
protein_data.head()

,h_F1_131N,h_F1_131C,h_F1_132C,h_F2_131N,h_F2_131C,h_F2_132C,h_F3_131N,h_F3_131C,h_F3_132C,h_F4_131N,...,s_F3_128N,s_F3_128C,s_F3_129C,s_F4_128N,s_F4_128C,s_F5_128N,s_F5_128C,s_F6_128N,s_F6_128C,s_F6_133N
Identifier,,,,,,,,,,,,,,,,,,,,,
P04114,0.750,0.853,0.822,1.191,1.175,1.078,0.693,0.947,0.931,1.057,...,1.044,1.305,1.657,1.323,1.624,1.170,0.981,0.791,1.029,1.195
P01024,0.782,1.057,0.994,0.864,0.917,0.790,0.823,1.152,0.816,0.920,...,1.100,0.986,1.114,1.210,1.289,1.104,1.111,1.007,1.159,0.979
P02768,1.183,1.101,1.045,1.086,1.041,1.187,1.234,1.079,1.011,1.099,...,0.786,0.706,0.947,0.831,0.717,0.795,0.776,0.938,0.903,0.743
P01023,1.066,1.278,0.959,0.811,0.789,0.931,0.971,0.769,1.011,0.866,...,0.817,0.728,0.861,0.798,0.751,0.917,0.809,0.780,1.195,0.706
P02751,1.085,0.947,0.993,1.343,1.130,0.778,0.731,1.084,1.107,0.909,...,0.566,0.854,1.109,0.630,0.850,0.661,0.848,0.829,0.760,0.811


In [8]:
protein_design

,group
sample,
h_F1_131N,healthy
h_F1_131C,healthy
h_F1_132C,healthy
h_F2_131N,healthy
h_F2_131C,healthy
...,...
s_F5_128N,severe
s_F5_128C,severe
s_F6_128N,severe


Read metabolomics data

In [9]:
compound_data = pd.read_csv(os.path.join(DATA_FOLDER, 'compound_data.csv'), index_col='Identifier')
compound_design = pd.read_csv(os.path.join(DATA_FOLDER, 'compound_design.csv'), index_col='sample')

In [10]:
compound_data.head()

,h_jkdz1,h_jkdz2,h_jkdz3,h_jkdz4,h_jkdz5,h_jkdz6,h_jkdz7,h_jkdz8,h_jkdz9,h_jkdz10,...,s_ZX12,s_ZX13,s_ZX14,s_ZX15,s_ZX16,s_ZX17,s_ZX18,s_ZX19,s_ZX20,s_ZX21
Identifier,,,,,,,,,,,,,,,,,,,,,
C21482,19413052.00,6.381812e+06,9.748316e+06,5.326872e+06,1.998072e+07,3.580375e+06,8.256121e+06,8.079382e+06,1.559659e+07,1.520363e+07,...,1.904349e+06,3.226016e+06,7.378147e+05,2.817698e+06,3.329101e+06,3206752.75,1.466174e+06,2.779301e+06,2.117668e+06,2.184310e+06
C18218,2711915.25,2.056393e+06,1.445594e+06,2.038765e+06,2.536996e+06,2.638198e+06,2.285757e+06,1.973140e+06,2.015425e+06,2.290842e+06,...,1.409720e+06,1.413307e+06,3.218834e+06,1.602131e+06,1.317878e+06,2930312.75,1.168094e+06,2.946776e+06,1.417311e+06,1.474166e+06
C05127,87727.25,NaN,9.238706e+04,NaN,1.597879e+05,NaN,NaN,NaN,9.055130e+04,1.214114e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.382788e+05,NaN,NaN
C01152,58832828.00,5.843934e+07,5.552133e+07,4.516214e+07,5.478952e+07,3.941259e+07,2.987876e+07,6.751726e+07,4.666031e+07,9.118524e+07,...,2.881314e+07,3.164358e+07,2.538767e+07,3.307604e+07,3.915698e+07,24400592.00,2.593375e+07,6.413868e+07,4.020588e+07,4.904488e+07
C02918,NaN,1.815549e+05,2.240392e+05,1.609397e+05,3.206194e+05,7.176557e+05,3.268182e+05,5.135810e+05,2.734582e+05,NaN,...,3.337245e+05,NaN,4.347152e+05,3.532118e+04,NaN,655827.25,8.359706e+05,4.034381e+06,2.839358e+05,8.062160e+04


In [11]:
compound_design

,group
sample,
h_jkdz1,healthy
h_jkdz2,healthy
h_jkdz3,healthy
h_jkdz4,healthy
h_jkdz5,healthy
...,...
s_ZX17,severe
s_ZX18,severe
s_ZX19,severe


In [12]:
set_log_level_info()

1

## Create a mapping object

The mapping object uses Reactome to map the different biological entities in the data:
- Transcripts (or genes) are connected to the proteins they encode
- Proteins and compounds are connected to reactions they're involved in
- Reactions are connected to pathways

In [13]:
m = Mapper(HOMO_SAPIENS) \
        .set_protein(protein_data, protein_design) \
        .set_compound(compound_data, compound_design) \
        .build()

2021-01-29 15:02:16.196 | INFO     | pyMultiOmics.functions:reactome_mapping:72 - Mapping genes -> proteins
2021-01-29 15:02:16.679 | INFO     | pyMultiOmics.functions:reactome_mapping:77 - Mapping proteins -> reactions
2021-01-29 15:02:17.464 | INFO     | pyMultiOmics.functions:reactome_mapping:85 - Mapping compounds -> reactions
2021-01-29 15:02:19.337 | INFO     | pyMultiOmics.functions:reactome_mapping:91 - Mapping reactions -> pathways
2021-01-29 15:02:19.633 | INFO     | pyMultiOmics.functions:reactome_mapping:102 - Mapping reactions -> proteins
2021-01-29 15:02:23.361 | INFO     | pyMultiOmics.functions:reactome_mapping:109 - Mapping reactions -> compounds
2021-01-29 15:02:24.736 | INFO     | pyMultiOmics.functions:reactome_mapping:121 - Mapping proteins -> genes
2021-01-29 15:02:34.019 | INFO     | pyMultiOmics.mapping:_add_nodes:206 - Processing nodes: genes
2021-01-29 15:02:34.546 | INFO     | pyMultiOmics.mapping:_add_nodes:206 - Processing nodes: proteins
2021-01-29 15:02:3

In [ ]:
m

## Query mapping object

Below shows some example queries we can perform with the mapping object

In [20]:
reactions = m.get_nodes(types=REACTIONS)

data = []
for reaction_id, reaction_data in reactions:
    reaction_name = reaction_data['display_name']
    proteins = m.get_connected(reaction_id, dest_type=PROTEINS, observed=True)
    compounds = m.get_connected(reaction_id, dest_type=COMPOUNDS, observed=True)
    
    if len(proteins) > 0 and len(compounds) > 0:
        row = [reaction_id, reaction_name, len(proteins), len(compounds)]
        data.append(row)

df = pd.DataFrame(data, columns=['reaction_id', 'reaction_name', 'num_proteins', 'num_compounds'])
df

,reaction_id,reaction_name,num_proteins,num_compounds
0,R-HSA-1605591,Glucosylceramidase cleaves the glucosidic bond...,1,1
1,R-HSA-163432,Cholesterol ester + h2o -> cholesterol + fatty...,1,1
2,R-HSA-163691,Phosphorylation of chrebp at serine 568 by ampk,1,1
3,R-HSA-1878002,Xylts transfer xyl to core protein,1,1
4,R-HSA-1889955,B3gat dimers transfer glca to tetrasaccharide ...,1,2
5,R-HSA-1889978,B3galt6 transfers gal to the tetrasaccharide l...,1,1
6,R-HSA-1889981,B4galt7 transfers gal group to xylosyl-unit of...,1,1
7,R-HSA-189384,"Blvra:zn2+, blvrb reduce bv to bil",1,2
8,R-HSA-194079,"Slco1b3 transports alb:(gcca, tcca) from extra...",1,1
9,R-HSA-194083,"Slco1b1 transports alb:(gcca,tcca) from extrac...",1,1


##### List all entities connected to reaction R-HSA-194153

In [21]:
query_id = 'R-HSA-194153'
m.get_connected(query_id)

,entity_id,display_name,data_type,observed
0,ENSG00000163631,Alb,genes,False
1,ENSG00000170231,Fabp6,genes,False
2,ENSG00000108846,Abcc3,genes,False
3,P02768,P02768,proteins,True
4,P51161,P51161,proteins,False
5,O15438,O15438,proteins,False
6,16525,Taurochenodeoxycholic acid,compounds,False
7,28865,Taurocholic acid,compounds,True
8,16359,Cholic acid,compounds,True
9,36274,Glycochenodeoxycholic acid,compounds,True


##### Query the connections between proteins and compounds (through their shared reactions)

In [24]:
query_id = 'P02768'
m.get_connected(query_id, dest_type=COMPOUNDS)

,entity_id,display_name,data_type,observed
0,16525,Taurochenodeoxycholic acid,compounds,False
1,28865,Taurocholic acid,compounds,True
2,16359,Cholic acid,compounds,True
3,36274,Glycochenodeoxycholic acid,compounds,True
4,30616,Atp(4-),compounds,False
5,9407,Taurochenodeoxycholate,compounds,True
6,456216,Adp(3-),compounds,False
7,17687,Glycocholic acid,compounds,False
8,16755,Chenodeoxycholic acid,compounds,True
9,43474,Hydrogenphosphate,compounds,False


In [25]:
query_id = '16990'
m.get_connected(query_id, dest_type=PROTEINS)

,entity_id,display_name,data_type,observed
0,Q9NPD5,Q9NPD5,proteins,False
1,P07148,P07148,proteins,False
2,P08263,P08263,proteins,False
3,P02768,P02768,proteins,True
4,P22309,P22309,proteins,False
5,Q9Y6L6,Q9Y6L6,proteins,False
6,P22310,P22310,proteins,False
7,P53004,P53004,proteins,False
8,P30043,P30043,proteins,True
9,P33527,P33527,proteins,False
